In [1]:
import requests
import datetime
import pandas as pd
import os
from contextlib import closing
import csv

In [2]:
start_date = (datetime.date.today() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
end_date = datetime.date.today().strftime("%Y-%m-%d")

In [52]:
start_time = start_date.replace('-', '') + '01'  # starting hours for each day
end_time = end_date.replace('-', '') + '24'  # last hour of each day
fmt = 'csv'
station = 240

url = "https://www.daggegevens.knmi.nl/klimatologie/uurgegevens/"
params = {
    "stns": station,
    "start": start_time,
    "end": end_time,
    "fmt": fmt
}

print(start_time)
print(end_time)

2021120401
2021120524


In [51]:
outname = f'knmi_raw.csv'
outdir = os.path.join('data', 'raw')
fullpath = os.path.join(outdir, outname)
commentpath = os.path.join(outdir, 'knmi_comment.txt')


with open(fullpath, 'w') as f:
    # create the csv writer
    writer = csv.writer(f)
    
    with requests.Session() as s:
        download = s.post(url, data=params, stream=True)
        download_iter = download.iter_lines() 
        
        # the following section deals with the comment header in the obtained csv file
        next_row = next(download_iter).decode('utf-8')
        header_row = next_row
        header_comment = ""
        
        #keep checking until we hit the first actual row
        while '#' in next_row:
            header_comment += next_row + '\n'
            header_row = next_row
            next_row = next(download_iter).decode('utf-8')
        
        with open(commentpath, "w") as text_file:
            text_file.write(header_comment)
        
        # the actual header is a comment, so we want to write that one
        writer.writerow(header_row[1:].split(','))
        writer.writerow(next_row.split(',')) # and this one bc the iterator hit it
            
        for line in download_iter:
            row = line.decode('utf-8')
            writer.writerow(row.split(','))


FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/knmi_raw.csv'